In [1]:
import findspark
findspark.init()
findspark.find()


'C:\\Users\\Admin\\anaconda3\\envs\\SparkEnvironment\\Lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
    .builder
    .appName("DataFrameApp")
    .master("local[4]")
    .getOrCreate()
)

sc = spark.sparkContext

spark

In [4]:
path = 'C:\\DataFiles\\new-data\\names.csv'

In [5]:


df = (
    spark
    .read
    .format("csv")
    .option("header", "true")
    .load(path)
)
df.show()
    

+---------+---------+
|firstname| lastname|
+---------+---------+
|    Rohit|   Sharma|
|    Virat|    Kohli|
|   Sachin|Tendulkar|
|   Zaheer|     Khan|
|  Rishabh|     Pant|
| Mohammed|    Shami|
|       KL|    Rahul|
+---------+---------+



In [13]:
df = df.withColumn("name", concat(col("lastname"), lit(","), col("firstname")))
df.show()

+---------+---------+----------------+
|firstname| lastname|            name|
+---------+---------+----------------+
|    Rohit|   Sharma|    Sharma,Rohit|
|    Virat|    Kohli|     Kohli,Virat|
|   Sachin|Tendulkar|Tendulkar,Sachin|
|   Zaheer|     Khan|     Khan,Zaheer|
|  Rishabh|     Pant|    Pant,Rishabh|
| Mohammed|    Shami|  Shami,Mohammed|
|       KL|    Rahul|        Rahul,KL|
+---------+---------+----------------+



In [6]:
# ---------- San Fransisco Fire Calls Case Study ------------------------
 
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
 StructField('UnitID', StringType(), True),
 StructField('IncidentNumber', IntegerType(), True),
 StructField('CallType', StringType(), True),
 StructField('CallDate', StringType(), True),
 StructField('WatchDate', StringType(), True),
 StructField('CallFinalDisposition', StringType(), True),
 StructField('AvailableDtTm', StringType(), True),
 StructField('Address', StringType(), True),
 StructField('City', StringType(), True),
 StructField('Zipcode', IntegerType(), True),
 StructField('Battalion', StringType(), True),
 StructField('StationArea', StringType(), True),
 StructField('Box', StringType(), True),
 StructField('OriginalPriority', StringType(), True),
 StructField('Priority', StringType(), True),
 StructField('FinalPriority', IntegerType(), True),
 StructField('ALSUnit', BooleanType(), True),
 StructField('CallTypeGroup', StringType(), True),
 StructField('NumAlarms', IntegerType(), True),
 StructField('UnitType', StringType(), True),
 StructField('UnitSequenceInCallDispatch', IntegerType(), True),
 StructField('FirePreventionDistrict', StringType(), True),
 StructField('SupervisorDistrict', StringType(), True),
 StructField('Neighborhood', StringType(), True),
 StructField('Location', StringType(), True),
 StructField('RowID', StringType(), True),
 StructField('Delay', FloatType(), True)])
 
 

In [7]:
path = 'C:\\DataFiles\\new-data\\sf-fire\\sf-fire-calls.csv'

In [8]:
df = (
    spark
    .read
    .format("csv")
    .option("header", "true")
    .load(path)
)
df.show(truncate = False)
    

+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+------------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+------------------------------+-------------------------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                       |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType      |UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood                  |Location                             |RowID        |Delay    |
+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+-------

In [24]:
df.filter(df.CallType != "Medical Incident").show(truncate = False)

+----------+------+--------------+-----------------------------+----------+----------+--------------------+----------------------+---------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+------------------------------+-------------------------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|CallType                     |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                    |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood                  |Location                             |RowID        |Delay    |
+----------+------+--------------+-----------------------------+----------+----------+--------------------+---------

In [46]:
df.filter(df.CallType.contains('Fire')).groupBy("CallType").agg(count("*").alias("count")).show(truncate = False)


+--------------+-----+
|CallType      |count|
+--------------+-----+
|Marine Fire   |14   |
|Vehicle Fire  |854  |
|Outside Fire  |2094 |
|Structure Fire|23319|
+--------------+-----+



In [39]:
df.select('CallType').distinct().show(truncate = False)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Confined Space / Structure Collapse         |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Polic

In [11]:
df.filter(df.Delay * 60 > 300 ).show(truncate = False)

+----------+------+--------------+-----------------------------+----------+----------+--------------------+----------------------+------------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+------------------------------+-------------------------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|CallType                     |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                       |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType      |UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood                  |Location                             |RowID        |Delay    |
+----------+------+--------------+-----------------------------+----------+----------+------------

In [16]:
df.groupBy("CallType").agg(count("*").alias("count")).sort(desc("count")).show(5)

+--------------------+------+
|            CallType| count|
+--------------------+------+
|    Medical Incident|113794|
|      Structure Fire| 23319|
|              Alarms| 19406|
|   Traffic Collision|  7013|
|Citizen Assist / ...|  2524|
+--------------------+------+
only showing top 5 rows



In [17]:
df.groupBy("Zipcode").agg(count("*").alias("count")).sort(asc("count")).show(truncate = False)

+-------+-----+
|Zipcode|count|
+-------+-----+
|NULL   |142  |
|94129  |512  |
|94158  |882  |
|94130  |1100 |
|94104  |1341 |
|94127  |1881 |
|94111  |2974 |
|94131  |3236 |
|94123  |3719 |
|94116  |3933 |
|94108  |4084 |
|94105  |4236 |
|94132  |4321 |
|94121  |4555 |
|94134  |5009 |
|94118  |5157 |
|94114  |5175 |
|94117  |5804 |
|94133  |6246 |
|94122  |6355 |
+-------+-----+
only showing top 20 rows

